# Compute statistics about the global database

* use lprofiles, ltags, ldeployments
* list files are created in sc_list_profiles.ipynb

In [5]:
from pathlib import Path
import sys
import os
import shutil
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
import gsw
from IPython.display import display,Image

from importlib import reload
import meop_metadata
import meop_plot_data
reload(meop_metadata)

<module 'meop_metadata' from '/media/disk2/roquet/MEOP_process/python/meop_metadata.py'>

## Write statistics

In [6]:
lprofiles, ltags, ldeployments = meop_metadata.read_lists_metadata(rebuild=False,verbose=False,public=False,Tdata=True)
lprofiles_public, ltags_public, ldeployments_public = meop_metadata.read_list_profiles(rebuild=False,verbose=False,public=True,Tdata=True)

In [7]:
lprofiles_public

,DEPLOYMENT_CODE,SMRU_PLATFORM_CODE,CYCLE_NUMBER,JULD,LATITUDE,LONGITUDE,N_TEMP,N_PSAL,N_CHLA,MASK
0,ft11,ft11-Cy07b-12,1,2012-02-12 22:19:59.999996,-49.778640,71.466896,27.0,27.0,18.0,Southern Ocean
1,ft11,ft11-Cy07b-12,2,2012-02-13 08:40:00.000001,-49.904131,71.869976,27.0,27.0,18.0,Southern Ocean
2,ft11,ft11-Cy07b-12,3,2012-02-13 21:39:59.999998,-50.050063,72.401875,27.0,27.0,18.0,Southern Ocean
3,ft11,ft11-Cy07b-12,4,2012-02-14 06:39:59.999998,-50.084059,72.901377,27.0,27.0,18.0,Southern Ocean
4,ft11,ft11-Cy07b-12,5,2012-02-14 18:39:59.999998,-50.093781,73.414020,27.0,27.0,18.0,Southern Ocean
...,...,...,...,...,...,...,...,...,...,...
644375,ct104,ct104-WF963-13,899,2014-10-25 04:07:00.000002,-73.397500,-111.908800,NaN,17.0,NaN,Southern Ocean
644376,ct104,ct104-WF963-13,900,2014-10-26 04:21:00.000002,-73.366200,-112.215100,NaN,17.0,NaN,Southern Ocean
644377,ct104,ct104-WF963-13,901,2014-10-26 11:29:00,-73.356800,-112.333800,NaN,16.0,NaN,Southern Ocean
644378,ct104,ct104-WF963-13,902,2014-10-26 12:22:00.000002,-73.355700,-112.348300,NaN,16.0,NaN,Southern Ocean


In [8]:
ltags.columns

Index(['SMRU_PLATFORM_CODE', 'DEPLOYMENT_CODE', 'JULD', 'LATITUDE',
       'LONGITUDE', 'N_PROF_TEMP', 'N_PROF_PSAL', 'N_PROF_CHLA', 'PUBLIC',
       'T1', 'T2', 'S1', 'S2', 'remove', 'Sremove', 'comment',
       'variable_offset', 'instr_id', 'year', 'period', 'continuous', 'prefix',
       'MASK'],
      dtype='object')

In [3]:
print('Number of valid T profiles, total :',lprofiles.N_TEMP.count())
print('Number of valid T profiles, public:',lprofiles_public.N_TEMP.count())
print('Number of valid T profiles, public, south of 55S: ',lprofiles_public[lprofiles_public.LATITUDE<-55].N_TEMP.count())

Number of valid T profiles, total : 657625
Number of valid T profiles, public: 590537
Number of valid T profiles, public, south of 55S:  272609


In [11]:
print('Number of valid S profiles, total :',lprofiles.N_PSAL.count())
print('Number of valid S profiles, public:',lprofiles_public.N_PSAL.count())
print('Number of valid S profiles, public, south of 55S: ',lprofiles_public[lprofiles_public.LATITUDE<-55].N_PSAL.count())

Number of valid S profiles, total : 521754
Number of valid S profiles, public: 457743
Number of valid S profiles, public, south of 55S:  234952


In [17]:
# select tags with at least one profile south of 55S
index_tags = lprofiles_public[lprofiles_public.LATITUDE<-55].SMRU_PLATFORM_CODE.unique()
print('Number of valid tags, public, south of 55S: ',len(index_tags))

Number of valid tags, public, south of 55S:  781


In [9]:
# select tags with at least one Chla profile
index_tags = lprofiles_public[lprofiles_public.N_CHLA>0].SMRU_PLATFORM_CODE.unique()
print('Number of valid tags, public, with chla: ',len(index_tags))

# select tags with at least one Chla profile south of 50S
index_tags = lprofiles_public[(lprofiles_public.N_CHLA>0)&(lprofiles_public.LATITUDE<-55)].SMRU_PLATFORM_CODE.unique()
print('Number of valid tags, public, with chla: ',len(index_tags))

# Number of profiles of Chla
print('')
print('Number of valid F profiles, total : ',lprofiles[lprofiles.N_CHLA>0].N_CHLA.count())
print('Number of valid F profiles, public: ',lprofiles_public[lprofiles_public.N_CHLA>0].N_CHLA.count())
print('Number of valid F profiles, public, south of 55S: ',lprofiles_public[(lprofiles_public.N_CHLA>0)&(lprofiles_public.LATITUDE<-55)].N_CHLA.count())

Number of valid tags, public, with chla:  108
Number of valid tags, public, with chla:  26

Number of valid F profiles, total :  26801
Number of valid F profiles, public:  26328
Number of valid F profiles, public, south of 55S:  3528
